In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
pip install gradio PyPDF2 langchain-google-genai langchain-core langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.8 MB/s eta 0:00:00


In [ ]:
!pip install sumy

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 14.9 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21692 sha256=691be35906806036a7a169f7d09f434b781e2dc8c413f9756d11523d26dc6268
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=8d4be4cdee00ac36ad5778229256f365c3b4ccfd506f65e979ac6de0e19ee127
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


In [ ]:
import gradio as gr
from PyPDF2 import PdfReader
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from transformers import T5ForConditionalGeneration, T5Tokenizer, BartForConditionalGeneration, BartTokenizer
import nltk
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
# Download nltk data
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Helper Functions
def extract_text_from_file(file_path):
    """Extract text from .txt or .pdf files."""
    if file_path.endswith(".txt"):
        with open(file_path, "r", encoding="utf-8") as file:
            return file.read()
    elif file_path.endswith(".pdf"):
        reader = PdfReader(file_path)
        return "".join(page.extract_text() for page in reader.pages)
    return ""


In [ ]:
def word_count(text):
    """Count the number of words in a text."""
    return len(text.split())

# Extractive Summarization
def frequency_summarize(text, max_words):
    stop_words = set(stopwords.words("english"))
    words = word_tokenize(text)
    freq_table = {word: words.count(word) for word in set(words) if word.lower() not in stop_words}
    sentences = sent_tokenize(text)
    sentence_scores = {sentence: sum(freq_table.get(word.lower(), 0) for word in word_tokenize(sentence)) for sentence in sentences}
    summary = " ".join(sentence for sentence, _ in sorted(sentence_scores.items(), key=lambda item: item[1], reverse=True)[:3])
    return summary[:max_words], word_count(text), min(max_words, len(summary.split()))

def lsa_summarize(text, max_words):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = " ".join(str(sentence) for sentence in summarizer(parser.document, 3))
    return summary[:max_words], word_count(text), min(max_words, len(summary.split()))

def lexrank_summarize(text, max_words):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = " ".join(str(sentence) for sentence in summarizer(parser.document, 3))
    return summary[:max_words], word_count(text), min(max_words, len(summary.split()))

def luhn_summarize(text, max_words):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LuhnSummarizer()
    summary = " ".join(str(sentence) for sentence in summarizer(parser.document, 3))
    return summary[:max_words], word_count(text), min(max_words, len(summary.split()))

# Abstractive Summarization
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small")
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")

bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

def t5_summarize(text, max_words):
    inputs = t5_tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = t5_model.generate(inputs, max_length=max_words, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True), word_count(text), max_words

def bart_summarize(text, max_words):
    inputs = bart_tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_model.generate(inputs, max_length=max_words, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
    return bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True), word_count(text), max_words

# Advanced Summarization (LLM)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)

def llm_summarize(text, num_words):
    prompt = ChatPromptTemplate.from_messages([("system", "Summarize this in {num_words} words:\n\n"), ("human", "{context}")])
    chain = prompt | llm
    result = chain.invoke({"context": text, "num_words": num_words})
    return result.content, word_count(text), num_words



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
import os
import asyncio
from typing import List
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from PyPDF2 import PdfReader
import gradio as gr

# Configure Google Gemini 1.5 Flash LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Utility to extract text from a PDF
def extract_text_from_pdf(pdf_path: str) -> List[str]:
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text.split("\n\n")  # Split into chunks (paragraphs)

# Iterative Refinement Components
summarize_prompt = ChatPromptTemplate([("human", "Write a concise summary of the following: {context}")])
initial_summary_chain = summarize_prompt | llm | StrOutputParser()

refine_template = """
Produce a final summary.

Existing summary up to this point:
{existing_answer}

New context:
------------
{context}
------------

Given the new context, refine the original summary.
"""
refine_prompt = ChatPromptTemplate([("human", refine_template)])
refine_summary_chain = refine_prompt | llm | StrOutputParser()

# Iterative Refinement Function
async def iterative_refinement(chunks: List[str]) -> str:
    summary = await initial_summary_chain.ainvoke(chunks[0], {})
    for chunk in chunks[1:]:
        summary = await refine_summary_chain.ainvoke(
            {"existing_answer": summary, "context": chunk}, {}
        )
    return summary

# Map-Reduce Components
map_prompt = ChatPromptTemplate([("human", "Write a concise summary of the following:\n\n{context}")])
map_chain = map_prompt | llm | StrOutputParser()

reduce_template = """
The following is a set of summaries:
{docs}
Take these and distill them into a final, consolidated summary of the main themes.
"""
reduce_prompt = ChatPromptTemplate([("human", reduce_template)])
reduce_chain = reduce_prompt | llm | StrOutputParser()

# Map-Reduce Function
async def map_reduce_summarization(chunks: List[str]) -> str:
    summaries = []

    # Step 1: Generate summaries for each chunk (Map Step)
    for chunk in chunks:
        summary = await map_chain.ainvoke(chunk, {})
        summaries.append(summary)

    # Step 2: Consolidate the summaries into a final summary (Reduce Step)
    consolidated_summaries = "\n".join(summaries)
    final_summary = await reduce_chain.ainvoke({"docs": consolidated_summaries}, {})
    return final_summary



In [3]:
import gradio as gr

# Define the Gradio Interface
with gr.Blocks(title="Summarization App") as demo:
    gr.Markdown("# Multi-Method Summarization App")

    with gr.Row():
        # Left Side - Input
        with gr.Column():
            gr.Markdown("## Inputs")
            input_text = gr.Textbox(label="Input Text", lines=10, placeholder="Enter or paste text here...")
            input_file = gr.File(label="Upload PDF File", file_types=[".pdf"])
            max_words = gr.Number(label="Max Words", value=50)

        # Right Side - Output
        with gr.Column():
            gr.Markdown("## Outputs")
            output_summary = gr.Textbox(label="Summary", lines=10, interactive=False)
            input_word_count = gr.Textbox(label="Input Word Count", interactive=False)
            output_word_count = gr.Textbox(label="Output Word Count", interactive=False)

    with gr.Row():
        # Summarization Type Selection
        summarization_type = gr.Radio(
            ["Extractive", "Abstractive", "Advanced"], label="Summarization Type"
        )

        # Method Selection
        summarization_method = gr.Dropdown(
            label="Choose Summarization Method",
            interactive=True
        )

        # Update Methods Dynamically
        def update_methods(s_type):
            if s_type == "Extractive":
                return gr.update(choices=["Frequency", "LSA", "LexRank", "Luhn"])
            elif s_type == "Abstractive":
                return gr.update(choices=["T5", "BART", "LLM"])
            elif s_type == "Advanced":
                return gr.update(choices=["LLM", "Iterative Refinement", "Map-Reduce"])
            return gr.update(choices=[])

        summarization_type.change(update_methods, inputs=[summarization_type], outputs=[summarization_method])

        # Summary Generation
        def generate_summary(s_type, method, text, file, max_words):
            # Load text from input source
            if file is not None:
                text = extract_text_from_file(file.name)

            if not text:
                return "Error: No text provided.", 0, 0

            # Call the appropriate summarization method
            if s_type == "Extractive":
                if method == "Frequency":
                    return frequency_summarize(text, max_words)
                elif method == "LSA":
                    return lsa_summarize(text, max_words)
                elif method == "LexRank":
                    return lexrank_summarize(text, max_words)
                elif method == "Luhn":
                    return luhn_summarize(text, max_words)
            elif s_type == "Abstractive":
                if method == "T5":
                    return t5_summarize(text, max_words)
                elif method == "BART":
                    return bart_summarize(text, max_words)
                elif method == "LLM":
                    return llm_summarize(text, max_words)
            elif s_type == "Advanced":


                if method == "LLM":
                    return llm_summarize(text, max_words)
                elif method == "Iterative Refinement":
                    pdf_path = pdf_file.name
                    chunks = extract_text_from_pdf(pdf_path)
                    text = extract_text_from_file(file.name)
                    return iterative_refinement(chunks),word_count(text),max_words
                elif method == "Map-Reduce":
                    pdf_path = pdf_file.name
                    chunks = extract_text_from_pdf(pdf_path)
                    return map_reduce_summarization(chunks),word_count(text),max_words


            return "Error: Invalid method.", 0, 0

        generate_button = gr.Button("Generate Summary")
        generate_button.click(
            generate_summary,
            inputs=[summarization_type, summarization_method, input_text, input_file, max_words],
            outputs=[output_summary, input_word_count, output_word_count],
        )

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://90a84f50af37073ac6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
